In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
top_500_urls = ["https://www.poemhunter.com/p/m/l.asp?a=0&l=top500&order=title&p=" + str(i) for i in range(1,21)]

url_stem = "https://www.poemhunter.com"
links = []

for url in top_500_urls:

    response = requests.get(url)
    soup = BeautifulSoup(response.text,"lxml")
    poems = soup.findAll('td',{"class":'title'})

    for poem in poems[1:]:
        links.append(url_stem + str(poem.find('a')['href']))

In [3]:
df = pd.DataFrame({"author" : [], "title": [], "text": []}, columns=["author","title","text"])

for link in links:
    try:
        #item = ["author", "title", "text"]
        response = requests.get(link)
        soup = BeautifulSoup(response.text,"lxml")
        text = soup.find('p',attrs={'class': None})
        text = str(text).replace("</p>","").replace("<p>","").replace("  "," ").replace('""','').strip()
        title_author = soup.find('title',attrs={'class': None})
        title_author = str(title_author).replace("</title>","").replace("<title>","").split("Poem by")
        title = title_author[0].strip()
        author = title_author[1].replace("-","").replace("Poem Hunter","").strip()
        df = df.append(pd.DataFrame({"author" : [author], "title": [title], "text": [text]}))
    except Exception as e:
        print(link)
        print(e)
df = df.reset_index(drop=True)

https://www.poemhunter.com/poem/after-auschwitz/
list index out of range
https://www.poemhunter.com/poem/i-sit-and-think/
list index out of range


In [12]:
df.head()
with open('poems_collection.csv', "w") as f:
    df.to_csv(f, header=True, index=False)